In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [ ]:
##load the dataset
max_features = 10000  # Vocabulary size
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
#print the shape of the data
print(f'Training data shape: {x_train.shape}, Training labels shape: {y_train.shape}')
print(f'Testing data shape: {x_test.shape}, Testing labels shape: {y_test.shape}')

17464789/17464789 [==============================] - 4s 0us/step
Training data shape: (25000,), Training labels shape: (25000,)
Testing data shape: (25000,), Testing labels shape: (25000,)


In [24]:
#inspect a sample review
sample_review = x_train[0]
sample_label = y_train[0]
print(f'Sample review (word indices): {sample_review}')
print(f'Sample label: {sample_label} (0=negative, 1=positive)')

Sample review (word indices): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Sample label: 1 (0=negative, 1

In [27]:
#mapping of words index back to words
word_index = imdb.get_word_index()
word_index
reversed_word_index = {value: key for (key, value) in word_index.items()}
decoded_review = ' '.join([reversed_word_index.get(i - 3, '?') for i in sample_review])
print(f'Decoded review: {decoded_review}')

Decoded review: ? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have do

In [ ]:
max_len = 500  # Maximum length of review
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]])

In [29]:
##train simple rnn model
model = Sequential()
model.add(Embedding(max_features,128, input_length=max_len)) #embedding layer
model.add(SimpleRNN(64,activation='relu'))  # Simple RNN layer
model.add(Dense(1, activation='sigmoid'))  # Output layer


In [30]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                12352     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1,292,417
Trainable params: 1,292,417
Non-trainable params: 0
_________________________________________________________________


In [31]:
#create an insitance of early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/20
157/157 [==============================] - 23s 140ms/step - loss: 0.6844 - accuracy: 0.6100 - val_loss: 0.6131 - val_accuracy: 0.6428
Epoch 2/20
157/157 [==============================] - 22s 142ms/step - loss: 0.5235 - accuracy: 0.7375 - val_loss: 0.5080 - val_accuracy: 0.7458
Epoch 3/20
157/157 [==============================] - 23s 143ms/step - loss: 0.4928 - accuracy: 0.8065 - val_loss: 0.4722 - val_accuracy: 0.7652
Epoch 4/20
157/157 [==============================] - 22s 143ms/step - loss: 1.3668 - accuracy: 0.8618 - val_loss: 0.4464 - val_accuracy: 0.7966
Epoch 5/20
157/157 [==============================] - 22s 138ms/step - loss: 0.2785 - accuracy: 0.9014 - val_loss: 0.4147 - val_accuracy: 0.8302
Epoch 6/20
157/157 [==============================] - 22s 139ms/step - loss: 0.1866 - accuracy: 0.9312 - val_loss: 0.3898 - val_accuracy: 0.8472
Epoch 7/20
157/157 [==============================] - 22s 141ms/step - loss: 0.1426 - accuracy: 0.9498 - val_loss: 0.4138 - val_ac

In [32]:
model.save('rnn_imdb_model.h5')
